In [1]:
import setup
setup.init()

In [2]:
from langgraph.checkpoint.memory import InMemorySaver 
from ai.agents import get_document_agent


{"success":true}


In [3]:
import uuid
# Initialize config
config = {
    "configurable": {
        "user_id": "3",
        "thread_id": str(uuid.uuid4()),
        "checkpoint_ns": ""  # Optional: add empty checkpoint_ns to satisfy checkpointer
    }
}
checkpointer = InMemorySaver()
agent = get_document_agent(config=config,checkpointer=checkpointer)

In [4]:
import uuid
import json
from ai.agents import get_document_agent, make_list_documents_tool, make_search_documents_tool, make_get_document_tool, make_create_document_tool, make_update_document_tool, make_delete_document_tool

# Initialize config
config = {
    "configurable": {
        "user_id": "3",
        "thread_id": str(uuid.uuid4())
    }
}

# Create agent
agent = get_document_agent(config=config)

# Create tools dictionary for manual execution
tools = {
    "list_documents": make_list_documents_tool(config),
    "search_documents": make_search_documents_tool(config),
    "get_document": make_get_document_tool(config),
    "create_document": make_create_document_tool(config),
    "update_document": make_update_document_tool(config),
    "delete_document": make_delete_document_tool(config)
}

# Combine messages and config into a single input dictionary
input_data = {
    "messages": [
        {"role": "user", "content": "I want you to create document about the Minions movie"}
    ],
    "configurable": config["configurable"]
}

# Manual agent invocation and tool execution
try:
    # Invoke the agent
    output = agent.invoke(input_data)
    print(f"Raw agent output: {output}")

    # Parse the output (handle ChatGroq response)
    if hasattr(output, 'content'):
        output = output.content
    if isinstance(output, dict):
        output = json.dumps(output)  # Convert dict to JSON string if necessary
    parsed_output = json.loads(output)

    # Check if it's a tool call or final answer
    action = parsed_output.get("action")
    if not action:
        raise ValueError("Output must contain an 'action' key")

    if action == "final_answer":
        print(f"Final answer: {parsed_output['output']}")
    else:
        # Execute the tool
        tool_name = action
        tool_input = parsed_output.get("action_input", {})
        if tool_name not in tools:
            raise ValueError(f"Unknown tool: {tool_name}")
        
        tool = tools[tool_name]
        tool_result = tool.invoke(tool_input)
        print(f"Tool result: {tool_result}")

        # Return the tool result as the final answer
        final_answer = {
            "action": "final_answer",
            "output": tool_result
        }
        print(f"Final answer: {final_answer['output']}")

except Exception as e:
    print(f"Error: {e}")

Error: Recursion limit of 25 reached without hitting a stop condition. You can increase the limit by setting the `recursion_limit` config key.
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/GRAPH_RECURSION_LIMIT


In [ ]:
response = agent.invoke(
    {
        "messages": [
            {"role": "user", "content": "Give me a summary of the Lord of the Rings movie."}
        ]
    },
    config = {
    "configurable": {
        "user_id": "3",
        "thread_id": str(uuid.uuid4()),
        "checkpoint_ns": ""  # Optional: add empty checkpoint_ns to satisfy checkpointer
    }
}
)
response['messages'][-1].content

In [ ]:
response = agent.invoke(
    {"messages": 
        [
            {"role": "user", "content": "I want you to create document about the Minions movie "}
        ]
    },
    config
)

response['messages'][-1].content

In [ ]:
response['messages'][-1].content

In [ ]:
response = agent.invoke(
    {"messages": 
        [
            {"role": "user", "content": "Update the Minions document content to include cast as well and year of release."}
        ]
    },
    config
)


In [ ]:
response = agent.invoke(
    {"messages": 
        [
            {"role": "user", "content": "Remove the Minions ."}
        ]
    },
    config
)

response['messages'][-1].content

In [ ]:
response = agent.invoke({"messages":
              [
                  {"role": "user", "content":"what are my recent documents saved?"}
              ]
            },
            config
        )

response['messages'][-1].content